# **Trajectory** back on original video

IDEA:
- take the processed positions
- apply the inverse of the homography
- print on the video frame by frame all points before the current frame

In [31]:
from pathlib import Path
import cv2
import csv

import numpy as np
import pandas as pd

In [ ]:
VIDEO_NUMBER = "7"
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(
    PROJECT_ROOT
    / "data"
    / f"recording_{VIDEO_NUMBER}"
    / f"Recording_{VIDEO_NUMBER}.mp4"
)
TRASFORMED_CSV_PATH = str(
    PROJECT_ROOT
    / "data"
    / "auxiliary_data"
    / "reconstructed_positions"
    / f"Transformed_positions_processed_{VIDEO_NUMBER}.csv"
)
LANE_CSV_PATH = str(
    PROJECT_ROOT
    / "data"
    / "auxiliary_data"
    / "lane_points"
    / f"Lane_points_{VIDEO_NUMBER}.csv"
)
OUTPUT_VIDEO_PATH = str(
    PROJECT_ROOT
    / "data"
    / f"recording_{VIDEO_NUMBER}"
    / f"Tracked_output_{VIDEO_NUMBER}.mp4"
)

In [33]:
# Load video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

In [34]:
# Load lane points and compute homographies
lane_points = pd.read_csv(LANE_CSV_PATH)
homographies = {}

width = 106  # Approximate pixels for 1.0668m
height = 1829  # Approximate pixels for 18.29m

for _, row in lane_points.iterrows():
    frame = int(row["Frame"])
    pts_src = np.array(
        [
            [row["bottom_left_x"], row["bottom_left_y"]],
            [row["bottom_right_x"], row["bottom_right_y"]],
            [row["up_right_x"], row["up_right_y"]],
            [row["up_left_x"], row["up_left_y"]],
        ],
        dtype=np.float32,
    )

    pts_dst = np.array(
        [[0, height], [width, height], [width, 0], [0, 0]], dtype=np.float32
    )

    H, _ = cv2.findHomography(pts_dst, pts_src)  # Inverse homography
    if H is not None:
        homographies[frame] = H

# Load real-world trajectory positions (top-down space)
positions = {}
with open(TRASFORMED_CSV_PATH, mode="r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for coord in reader:
        frame_num, x, y = coord
        frame_num = int(frame_num)
        if x and y:
            positions[frame_num] = (float(x), float(y))  # Store in top-down space
        else:
            positions[frame_num] = None

In [ ]:
# Draw trajectory frame-by-frame
frame_count = 0
trajectory = []  # Store real-world trajectory (top-down)

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Collect current position if available
    if frame_count in positions and positions[frame_count] is not None:
        trajectory.append(positions[frame_count])

    # Transform all points in trajectory to current frame using current frame's homography
    refreshed_trajectory = []
    if frame_count in homographies:
        H = homographies[frame_count]
        for x, y in trajectory:
            transformed = cv2.perspectiveTransform(
                np.array([[[x, y]]], dtype=np.float32), H
            )[0][0]
            refreshed_trajectory.append((int(transformed[0]), int(transformed[1])))

    # Draw trajectory lines
    for i in range(1, len(refreshed_trajectory)):
        cv2.line(
            frame, refreshed_trajectory[i - 1], refreshed_trajectory[i], (0, 0, 255), 2
        )

    # Draw current ball position
    if refreshed_trajectory:
        x, y = refreshed_trajectory[-1]
        cv2.circle(frame, (x, y), 4, (0, 0, 255), -1)  # Red dot

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
print(f"Tracking video saved to {OUTPUT_VIDEO_PATH}")

End of video or error reading frame.
Tracking video saved to /home/davic/projects/IACV_project/bowling-analysis/data/recording_7/Tracked_output_7.mp4
